## 协程

相较于Python的其他特性，协程的资料较少（在阅读这一章之前，本人之前从未有意识的使用过）。从句法上来看协程与生成器类似，这两者均是包含yield关键字的“函数”。不同的是，协程中yield通常出现在表达式的右侧 —— 可以产出值也可以不产出值。协程可能会从调用方接收数据，这一过程由调用方主动使用.send()方法提供

从功能上来看，协程中的yield关键字是一种流程控制工具，以实现协作式多任务。

## 协程的基本行为

Python中协程的正常运作依赖于生成器
1. 协程使用生成器函数定义，即定义体中包含yield关键字
2. 对于协程，yield关键字在表达式中使用，并且能够从调用方接收数据，默认值为None
3. 协程的运作方式类似于生成器 —— 首先需要得到生成器对象，并且使用next()函数调用该对象后会运行到第一个yield语句
4. 当程序运行到定义体末尾后，生成器同样会抛出StopIteration

In [22]:
from inspect import getgeneratorstate

def simple_coroutine():
    print("-> coroutine started")
    x = yield
    print("-> coroutine received: ", x)

my_coro = simple_coroutine()
print(type(my_coro))
print(getgeneratorstate(my_coro))
next(my_coro)
print(getgeneratorstate(my_coro))
my_coro.send(1024)

<class 'generator'>
GEN_CREATED
-> coroutine started
GEN_SUSPENDED
-> coroutine received:  1024


StopIteration: 

上述例子表明协程可能处于多种状态，这些状态可以由inspect.getgeneratorstate()函数获得：
* GEN_CREATED：等待开始执行
* GEN_RUNNING：解释器正在执行
* GEN_SUSPENDED：在yield表达式处暂停
* GEN_CLOSED：执行结束

值得注意的是，仅当生成器处于GEN_SUSPENDED状态时才可以使用.send()传递数据。若在GEN_CREATED状态时传递则会抛出TypeError: can't send non-None value to a just-started generator；若在GEN_CLOSED阶段传递则会抛出StopIteration。

### 协程的执行过程

协程的执行过程需要着重理解。关键在于暂停的位置以及.send()传递数据的时机。

实际上也就一点：对于生成器，若调用next()函数，则**该生成器会运行到下一个yield语句处（包含yield语句），.send()语句传递数据在此之后**

下述例子描述了协程的基本执行过程，本书的图16-1对协程的执行过程有很清晰的描述

In [7]:
def sample_coro2(a):
    print("-> started: a = : ", a)
    b = yield a
    print("-> Received: b = : ", b)
    c = yield a + b
    print("-> Received: c = : ", c)

my_coro2 = sample_coro2(14)
print("开始执行")
print(next(my_coro2))
print("第一次send：")
print(my_coro2.send(28))
print("第二次send：")
print(my_coro2.send(99))

开始执行
-> started: a = :  14
14
第一次send：
-> Received: b = :  28
42
第二次send：
-> Received: c = :  99


StopIteration: 

## 预激协程

协程使用前需要进行预激，即让生成器处于GEN_SUSPENDED状态。

预激协程最为朴素的方法就是通过next()函数，使得生成器运行到第一个yield位置。为了简化起见，还可以通过装饰器预激协程。具体来说，通过实现一个装饰器自动运行next()函数。

用于预激协程的装饰器需要自行实现，本书给出了一个例子

In [8]:
from functools import wraps

def coroutine(func):
    @wraps(func)
    def primer(*args, **kwargs):
        # 创建生成器
        gen = func(*args, **kwargs)
        # 预激
        next(gen)
        # 返回生成器
        return gen
    return primer

@coroutine
def averager():
    total = 0.0
    count = 0
    average = None
    while True:
        term = yield average
        total += term
        count += 1
        average = total/count

coro_avg = averager()
print(coro_avg.send(10))
print(coro_avg.send(30))
print(coro_avg.send(50))

10.0
20.0
30.0


## 终止协程和异常处理

协程中没有处理的异常会向上冒泡至调用方，并且终止该协程。

上述过程实际上暗示了某种终止协程的方法：通过传递某些特殊的数据，让协程报错后自动终止。例如传递None、StopIteration等

更“温和”的做法是通过两个方法显式的传递异常给协程：
1. throw
.throw(exc_type[, exc_value[, traceback]])，显式传递错误给协程，若协程处理了该错误则会自动运行到下一个yield位置并正常返回产出的值；否则该异常会向上冒泡至调用方
2. close
强制协程在暂停处抛出GeneratorExit，若协程没有处理该错误则协程终止并且调用方不会报错

close方法可以终止协程，throw方法则可以显式的传递特定错误给协程

## 协程的返回值

协程可以使用return返回值，这一特性是Python3.3新增的，对于Python3.3之前的版本，若同时使用yield和return会抛出句法错误。

具体来说，当协程正常终止后（抛出StopIteration），return的内容会自动赋值到StopIteration的一个属性，通过处理该错误能够得到返回值。下述是一个例子。该例子中，当传递None给协程时，该协程会退出while循环并且返回一个包含均值和项数的字典，在主程序里通过try/except捕捉StopIteration错误能够得到返回值。

In [2]:
def averager():
    total = 0.0
    count = 0
    average = None
    while True:
        term = yield
        if term is None:
            break
        total += term
        count += 1
        average = total / count
    return {"count": count, "average": average}

coro_avg = averager()
next(coro_avg)
coro_avg.send(10)
coro_avg.send(30)
coro_avg.send(50)
try:
    coro_avg.send(None)
except StopIteration as exc:
    result = exc.value
    print(result)

{'count': 3, 'average': 30.0}


## 协程中的yield from

上述获取返回值的方法略别扭，好比在while循环中手动使用迭代器获取元素，完了还需要处理StopIteration。与for循环会自动生成并使用迭代器一样，yield from会自动处理StopIteration并返回结果。

为了描述yield from，PEP380中使用了三个术语：

1. 委派生成器
    包含yield from <iterable>表达式的生成器函数
2. 子生成器
    从yield from <iterable>的<iterable>中获取的生成器
3. 调用方
    调用委派生成器的主程序

本书的图16-2描述了上述三个术语之间的关系以及数据/消息在这些对象之间的传递：

调用方能够使用.send、.throw和.close传递数据并控制委派生成器，委派生成器能够返回数据给调用方；委派生成器则可以进一步控制子生成器，同时委派生成器也会设法处理子生成器抛出的错误。

下面是使用yield from的一个例子

In [27]:
from inspect import getgeneratorstate

# 子生成器
def averager():
    total = 0.0
    count = 0
    average = None
    while True:
        term = yield
        if term is None:
            break
        total += term
        count += 1
        average = total/count
    return {"count": count, "average": average}


# 委派生成器
def grouper(results, key):
    while True:
        results[key] = yield from averager()    


# 调用方
def main():
    test_data = {"group_1": [10, 30, 50, 90, 100], "group_2":[8, 10, 9, 7, 19]}
    results = {}
    for key, values in test_data.items():
        group = grouper(results, key)
        next(group)
        for value in values:
            group.send(value)
        group.send(None)
    print(results)

main()

{'group_1': {'count': 5, 'average': 56.0}, 'group_2': {'count': 5, 'average': 10.6}}


上述程序执行了相当多的操作，对于test_data中的每一个键值对：

1. 新建grouper实例group，并传入记录数据的results和键key。group是一个委派生成器
2. 使用next(group)预激委派生成器并使得子生成器在yield暂停，同时group也在yield from位置暂停
3. 逐个传入数据，调用方传递数据给group，group传递数据给子生成器。值得注意的是虽然子生成器在获取到数据后会运行到下一个yield位置，group依然停留在yield from的位置并没有运行委派生成器里的赋值语句
4. 调用方的内层for循环执行完毕后，发送None给group，group传递None给子生成器，子生成器抛出StopIteration并执行return；group处理StopIteration并运行委派生成器中的赋值语句，然后**运行至下一个yield from位置**
5. 运行调用方的外层for循环，并重新执行上述1~4步。前一个group实例以及其子生成器被自动回收

yield from的行为可以归纳为如下几点：
1. 子生成器产出的值均会直接传递给调用方
2. 调用方使用send()传递给委派生成器的值均会直接传递给子生成器。若传递的值不是None则调用子生成器的send()方法，否则调用__next__()方法。若该调用抛出StopIteration，则委派生成器恢复运行，任何其他异常均会向上冒泡传递给委派生成器
3. 子生成器退出时，子生成器的return expr会触发StopIteration(expr)
4. yield from表达式的值是子生成器终止时传给StopIteration异常的第一个参数
5. 调用方传递给委派生成器的异常，除了GeneratorExit外，均会传递给子生成器的throw()方法。若调用throw()方法抛出StopIteration，则委派生成器恢复运行；其他异常则会由子生成器向上冒泡至委派生成器
6. 若调用方传递GeneratorExit异常给委派生成器，或者调用close()方法。若子生成器有close()方法则调用该方法。若调用close()方法导致异常，则向上冒泡该错误至委派生成器；否则委派生成器抛出GeneratorExit